# Task 6

### Find fans/lovers. The idea is, if the user likes another posts, and its is not mutual, then the first fan/lover

In [1]:
import pyspark
import pyspark.sql.functions
import os

In [2]:
# Configuration of SPARK_HOME system variable
os.environ["SPARK_HOME"] = "C:/spark"

In [3]:
# pyspark session and SQL context creation
configs = pyspark.SparkConf().setAppName("task6").setMaster("local")
spark_context = pyspark.SparkContext(conf=configs)
spark = pyspark.sql.SparkSession(spark_context)
sql_context = pyspark.sql.SQLContext(spark_context)

In [10]:
# Followers posts likes data loading, clearing from 
#'self likes' and repetitions in ownerId - likerId
data_followers_posts_likes = spark.read.load("followers_posts_likes.parquet/*.parquet").\
    select("ownerId", "likerId").\
    where(pyspark.sql.functions.col("ownerId") != pyspark.sql.functions.col("likerId")).distinct()

# Fans data creation: (id_user, Array (his_fan_ids))
data_fans = data_followers_posts_likes.alias("t1").\
    join(data_followers_posts_likes.alias("t2"), 
         (pyspark.sql.functions.col("t1.ownerId") == pyspark.sql.functions.col("t2.likerId")) &
         (pyspark.sql.functions.col("t1.likerId") == pyspark.sql.functions.col("t2.ownerId")), 
         "left_outer").\
    where(pyspark.sql.functions.col("t2.likerId").isNull()).\
    select(pyspark.sql.functions.col("t1.ownerId").alias("id_user"), 
           pyspark.sql.functions.col("t1.likerId").alias("id_fan")).\
    sort("id_fan").groupby("id_user").agg(pyspark.sql.functions.collect_list("id_fan")).sort("id_user")

# Fans data showing
data_fans.show()

# Fans data saving
data_fans.coalesce(1).write.format("json").save("task6_output.json")

+-------+--------------------+
|id_user|collect_list(id_fan)|
+-------+--------------------+
|    637|[1217, 1255, 1751...|
|   1087|[1818, 4177, 6308...|
|   1119|[1099, 4836, 2237...|
|   1127|[1567, 1679, 2282...|
|   1174|[7627, 19755, 272...|
|   1567|[1099, 1113, 1119...|
|   1632|[42940, 68535, 75...|
|   1669|[1216, 6107, 7068...|
|   2212|[1099, 1405, 2767...|
|   2718|[1105, 15275, 409...|
|   2976|[2541, 3448, 5304...|
|   3420|[1543, 1568, 2050...|
|   3768|[25068229, 386898...|
|   3868|[19249, 23678, 26...|
|   3972|[1410, 34660, 482...|
|   4023|[4734, 21272, 283...|
|   4107|[6707, 7439, 3349...|
|   4990|[277835, 286893, ...|
|   5630|[105815, 7696061,...|
|   5648|[1614, 1991, 3757...|
+-------+--------------------+
only showing top 20 rows

